In [436]:
import io
import os
from typing import Dict, Any
import logging
from collections import OrderedDict
import json
import glob

import pandas as pd
import numpy as np
import PyPDF2


In [437]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [438]:
def extract_pdf_fields(pdf_bytes: bytes) -> Dict[str, Any]:
    """
    Extract form fields from a fillable PDF (AcroForm).
    
    Args"
        pdf_bytes: PDF file as bytes
        
    Returns:
        Dictionary of field names and values
    """
    try:
        pdf_file = io.BytesIO(pdf_bytes)
        reader = PyPDF2.PdfReader(pdf_file)
        
        fields = {}
        
        # Try to extract form fields
        if reader.get_fields():
            for field_name, field_data in reader.get_fields().items():
                value = field_data.get('/V', '')
                if isinstance(value, bytes):
                    value = value.decode('utf-8', errors='ignore')
                fields[field_name] = str(value)
        else:
            logger.warning("No form fields found in PDF - may need OCR")
            # Return basic metadata
            fields['_pages'] = len(reader.pages)
            fields['_has_form_fields'] = False
            
        return fields
        
    except Exception as e:
        logger.error(f"Error extracting PDF fields: {e}")
        return {'_error': str(e)}

In [439]:
def get_spreadsheet_data(pdf_fields: Dict[str, Any]) -> Dict[str, Any]:
    """
    Extract form fields from a fillable PDF (AcroForm).
    
    Args"
        pdf_bytes: PDF file as bytes
    """
    global SPREADSHEET_FIELD_NAMES
    global FIELD_NAME_MAP
    matching_field_names = [f for f in SPREADSHEET_FIELD_NAMES if f in list(pdf_fields.keys())]
    non_matching_field_names = [f for f in SPREADSHEET_FIELD_NAMES if f not in list(pdf_fields.keys())]
    try:
        data_ls = []
        for f in SPREADSHEET_FIELD_NAMES:
            if f in matching_field_names:
                data_ls.append(f"{f}: {pdf_fields[f]}")
            elif f in non_matching_field_names:
                if f == 'Other Specialty Contractors':
                    data_ls.append(f"{f}:")
                else:
                    data_ls.append(f"{f}: {pdf_fields[FIELD_NAME_MAP[f]]}")
            else:
                data_ls.append(f"{f}: ''")
    except Exception:
        return None
    return data_ls

In [440]:
SPREADSHEET_FIELD_NAMES = [
    "Owner", "Architect", "General Contractor", "Civil Engineer", "Electrical Engineer", 
    "Mechanical Engineer", "Structural Engineer", "Interior Design", "Landscape Design", 
    "Geotech", "Structural Concrete", "GFRC Panels & Precast", "Plumbing", "HVAC", 
    "Electrical", "Masonry", "Tile/Stone", "Glass/Curtain Wall", "Flooring", "Roofing", 
    "Steel", "Excavation", "Metal Framing and Drywall", "Other Specialty Contractors" 
]

FIELD_NAME_MAP = {
        'Civil Engineer': 'Civil',
        'Electrical Engineer': 'Electrical',
        'Mechanical Engineer': 'Mechanical',
        'Structural Engineer': 'Structural',
        'Structural Concrete': 'Concrete',
        'GFRC Panels & Precast': 'Precast',
        'Tile/Stone': 'TileStone',
        'Glass/Curtain Wall': 'GlassCurtain Wall',
        'Steel': 'Steel Fabrication',
        'Metal Framing and Drywall': 'DrywallAcoustics',
        'Landscape Design': 'Landscape Architect',
    }

# Data Setup

### Get project categories and winners

In [441]:
winners_df = pd.read_excel('Clean2fara_11.11.25.xlsx')
winners_df = winners_df.rename(columns={'Unnamed: 0': 'Project #'})
category_map = dict(zip(winners_df['Project Name'], winners_df['Category']))

In [442]:
winners_df.head()

,Project #,Category,Project Name,Submitted Project Name,Submitting Firm(s)
0,1,Project of the Year,Black Desert Resort,Black Desert Resort,SIRQ Construction
1,2,Adaptive Reuse,Arbor 515,Arbor 515 (515 Tower Phase 1),Architecture Belgique / Bonneville Builders
2,3,Commercial/Financial,Mountain America Credit Union West Valley Branch,Mountain America Credit Union West Valley Branch,City Creek Construction
3,4,Commercial/Mixed-Use,Downtown Daybreak,Downtown Daybreak,Okland Construction
4,5,Commercial/Office,Wavetronix Springs,Wavetronix Springs,FFKR Architects / Jacobsen Construction


project_categories_df = pd.read_csv('ProjectCategories.csv')
project_categories = dict(zip(project_categories_df['Order'], project_categories_df['ProjectCategory']))

### Normalize project names

In [443]:
dataframes_ls = []
folders = os.listdir('data')
for folder in folders:
    pdf_file_ls = glob.glob(f'data/{folder}/*.pdf')
    if len(pdf_file_ls) != 1:
        raise ValueError(f'{folder} has {len(pdf_file_ls)} pdf files')
    pdf_bytes = open(pdf_file_ls[0], 'rb').read()
    pdf_fields = extract_pdf_fields(pdf_bytes)
    spreadsheet_data = get_spreadsheet_data(pdf_fields)
    if not spreadsheet_data:
        print(f'{pdf_file_ls[0]} has no data')
        continue
    official_name = pdf_fields['Official Name'].lstrip()
    if official_name not in winners_df['Project Name'].values:
        print(f"{official_name} not in winners_df['Project Name'].values")
        continue

Arbor 515 (515 Tower Phase 1) not in winners_df['Project Name'].values
Snowbird Resort - Mid Gad not in winners_df['Project Name'].values
Sandy City Fire Station #31 not in winners_df['Project Name'].values
JP Morgan Chase One Utah Center  not in winners_df['Project Name'].values
Wasatch Fire District Station 51 (Wasatch Fire District, Claude R. and June M. Hicken Fire Station 51) not in winners_df['Project Name'].values
Provo Water Center not in winners_df['Project Name'].values
Salt Lake City International Airport Terminal Redevelopment Program Phases 3 and 4 (Design Package 20), Paving and Walls not in winners_df['Project Name'].values
Ogden Eccles Conference Center  not in winners_df['Project Name'].values
Utah State University Gardner Learning & Leadership Building  not in winners_df['Project Name'].values
Utah Hockey Club Practice Facility not in winners_df['Project Name'].values
data/30.The Well Church/UCD Application - The Well Church.pdf has no data


### Normalize field names

Get field names for the first PDF.

In [444]:
dataframes_ls = []
folders = os.listdir('data')
for folder in folders:
    pdf_file_ls = glob.glob(f'data/{folder}/*.pdf')
    if len(pdf_file_ls) != 1:
        raise ValueError(f'{folder} has {len(pdf_file_ls)} pdf files')
    pdf_bytes = open(pdf_file_ls[0], 'rb').read()
    pdf_fields = extract_pdf_fields(pdf_bytes)
    spreadsheet_data = get_spreadsheet_data(pdf_fields)
    if not spreadsheet_data:
        print(f'{pdf_file_ls[0]} has no data')
        continue
    official_name = pdf_fields['Official Name'].lstrip()
    submitting_firm = pdf_fields['Name of Firm']
    winner_info = winners_df[winners_df['Submitted Project Name'] == official_name]
    if len(winner_info) != 1:
        raise ValueError(f'{official_name} has {len(winner_info)} winner info')
    category = winner_info['Category'].values[0]
    project_number = winner_info['Project #'].values[0]
    project_name = winner_info['Project Name'].values[0]
    df = pd.DataFrame({
        'Project #': [project_number, project_number] + [''] * (len(spreadsheet_data) - 2),
        'Category & Project Name': [category, project_name] + [''] * (len(spreadsheet_data) - 2),
        'Submitting Firm(s)': [submitting_firm] + [''] * (len(spreadsheet_data) - 1),
        'Project Team': spreadsheet_data
    })
    empty_rows_df = pd.DataFrame(np.nan, index=[0, 1], columns=df.columns)
    df = pd.concat([df, empty_rows_df], ignore_index=True)
    df['group_number'] = project_number
    df['row_number'] = df.index
    dataframes_ls.append(df)
    
    

data/30.The Well Church/UCD Application - The Well Church.pdf has no data


folders = os.listdir('data')
# Use an ordered dict and iterate sequentially by key
folders_dict = OrderedDict()
for folder_name in folders:
    parts = folder_name.split('.', 1)
    try:
        num = int(parts[0])
        folders_dict[num] = folder_name
    except ValueError:
        continue
dataframes_ls = []
for key in sorted(folders_dict.keys()):
    category = project_categories[key]
    folder = folders_dict[key]
    pdf_file_ls = glob.glob(f'data/{folder}/*.pdf')
    if len(pdf_file_ls) != 1:
        raise ValueError(f'{folder} has {len(pdf_file_ls)} pdf files')
    for pdf_file in pdf_file_ls:
        pdf_bytes = open(pdf_file_ls[0], 'rb').read()
        pdf_fields = extract_pdf_fields(pdf_bytes)
        data_dict, data_ls, owner, submitting_firms, project_name = parse_pdf_fields(pdf_fields)
        if not data_ls:
            dataframes_ls.append(
                pd.DataFrame({
                    'Project #': [np.nan, np.nan],
                    'Category & Project Name': [np.nan, np.nan],
                    'Submitting Firm(s)': [np.nan, np.nan],
                    'Project Team': [np.nan, np.nan]
                })
            )
            print(f'{pdf_file_ls[0]} has no data')
            continue
        df = pd.DataFrame({
            'Project #': [key, key] + [''] * (len(data_ls) - 2),
            'Category & Project Name': [category, project_name] + [''] * (len(data_ls) - 2),
            'Submitting Firm(s)': [submitting_firms] + [''] * (len(data_ls) - 1),
            'Project Team': data_ls
        })
        empty_rows_df = pd.DataFrame(np.nan, index=[0, 1], columns=df.columns)
        df = pd.concat([df, empty_rows_df], ignore_index=True)
        dataframes_ls.append(df)

In [445]:
output_df = pd.concat(dataframes_ls).sort_values(by=['group_number', 'row_number']).drop(columns=['group_number', 'row_number'])

In [446]:
output_df.to_excel('0.Project Team Sheet_11.11.25 - Sheet 1.xlsx', index=False)